# Explore Model Instance
After a model is run, this notebook can be used to explore the model instance and results, if exported as pickle files.

To export the model instance as a pickle file, `--save-instance` must be added to `options.txt` (NOTE: this will be a large file!)  
Switch will automatically export the model results as a pickle file unless the `--no-save-solution` is specified in `options.txt`  

In order to explore duals and reduced costs, `-suffixes dual rc` must be added to `options.txt`

In [8]:
from pathlib import Path
import pickle
import cloudpickle
import pyomo.environ as pyo
import pandas as pd

# specify where the pickle file is located
model_path = '../MODEL_RUNS/generic_office_example/outputs/annual_goal/'

# Explore Model Instance

In [2]:
# read instance file
with open((Path.cwd() / model_path / 'instance.pickle'), mode='rb') as file:
    instance = cloudpickle.load(file)

In [13]:
len(instance.GENERATION_PROJECTS)

9

## Explore Sets

In [ ]:
i = 0
for setobject in instance.component_objects(pyo.Set, active=True):
    nametoprint = str(str(setobject.name))
    if '_index' in nametoprint:
        pass
    else:
        print ("Set ", nametoprint)  
        i+=1
print(f'Total Number of sets: {i}')

In [ ]:
# to examine the set
instance.GENS_BY_TECHNOLOGY.pprint()

## Explore Parameters, Expressions, and Vars

In [ ]:
for ExpObject in instance.component_objects(pyo.Expression, active=True):
    nametoprint = str(str(ExpObject.name))
    print ("Expression ", nametoprint) 

In [ ]:
instance.DLAPLoadCostInTP.pprint()

In [ ]:
pyo.value(instance.VariableGen['PV_Kern', 12])

In [ ]:
# print all parameters
i=0
for parmobject in instance.component_objects(pyo.Param, active=True):
    nametoprint = str(str(parmobject.name))
    print ("Parameter ", nametoprint)  
    i+=1
print(f'Total Number of parameters: {i}')

In [ ]:
instance.add_one_to_period_end.pprint()

## Explore Constraints
I might not be able to access constraint values: https://stackoverflow.com/questions/50703321/how-to-retrieve-value-of-constraint-from-pyomo

In [ ]:
# list all of the constraints
for c in instance.component_objects(pyo.Constraint, active=True):
    print ("Constraint",c)

In [ ]:
instance.Enforce_Hourly_Renewable_Target.pprint()

In [ ]:
# can get numerical values of single constraint through lower, body, upper
c = instance.Zone_Energy_Balance
print ("   Constraint",c)
i = 1
for index in c:
    if i < 5:
        print ("      ", index, c[index].upper)
        i+=1
    else:
        break

## Explore Slack Values

In [ ]:
# https://pyomo.readthedocs.io/en/stable/working_models.html#accessing-slacks

c = instance.Zone_Energy_Balance
print ("   Constraint",c)
for index in c:
    print ("      ", index, c[index].lslack(), round(c[index].uslack(),4))


# Explore Results

The SolverResults object contains four main pieces of data:
 - Problem
 - Solver
 - Solution
 - Pyomo solve time

In [9]:
# read results file
with open((Path.cwd() / model_path / 'results.pickle'), 'rb') as file:
    results = pickle.load(file)

print(results.problem)


- Name: unknown
  Lower bound: 726748.0428
  Upper bound: 726748.0428
  Number of objectives: 1
  Number of constraints: 43803
  Number of variables: 35051
  Number of binary variables: <undefined>
  Number of integer variables: <undefined>
  Number of continuous variables: <undefined>
  Number of nonzeros: 17479
  Sense: minimize



In [3]:
print(results.solver)


- Name: <undefined>
  Status: ok
  Return code: <undefined>
  Message: <undefined>
  User time: -1.0
  System time: 28.21
  Wallclock time: 28.21
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created subproblems: None
    Black box: 
      Number of function evaluations: <undefined>
      Number of gradient evaluations: <undefined>
      Number of iterations: 33511
  Error rc: 0
  Time: 28.228245973587036



In [10]:
print(results.pyomo_solve_time)

26.032098054885864


In [ ]:
print(results.solution)

## Explore Reduced Costs

In [ ]:
variable_rc = pd.DataFrame.from_dict(results.solution.Variable, orient='index')
variable_rc = variable_rc.reset_index()
variable_rc[['Variable','index']] = variable_rc['index'].str.split('[', expand=True)
variable_rc['index'] = '[' + variable_rc['index']
variable_rc = variable_rc[['Variable','index','Value','Rc']]
variable_rc

## Explore Duals

In [ ]:
constraint_duals = pd.DataFrame.from_dict(results.solution.Constraint, orient='index')
constraint_duals = constraint_duals.reset_index()
constraint_duals[['Variable','index']] = constraint_duals['index'].str.split('[', expand=True)
constraint_duals['index'] = '[' + constraint_duals['index']
#constraint_duals = constraint_duals[['Variable','index','Value','Rc']]
constraint_duals